In [1]:
%env FILEPATH_FEATURE_SWITCH = TRUE
%env DJ_SUPPORT_FILEPATH_MANAGEMENT = TRUE

env: FILEPATH_FEATURE_SWITCH=TRUE
env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE


In [2]:
import datajoint as dj
from churchland_pipeline import acquisition as acq
from churchland_pipeline import action      as act
from churchland_pipeline import ephys       as eph
from churchland_pipeline import lab
from churchland_pipeline import processing  as pro
from churchland_pipeline import reference   as ref

from churchland_pipeline.tasks import pacman as pac
from churchland_pipeline import common as com

Connecting ChurchlandLab_test@localhost:3306


In [3]:
import os
import re
from datetime import datetime

In [4]:
# monkey and rigs to import
MONKEY = 'Cousteau'
RIG = 'Jumanji'

In [5]:
# session dates
rawPath = '/srv/locker/churchland/{}/pacman-task/{}/raw/'.format(RIG, MONKEY.lower())
dates = sorted(list(os.listdir(rawPath)))
dates = [x for x in dates if re.search('\d{4}-\d{2}-\d{2}',x) is not None]

In [6]:
# import session data
for date in dates:
    if (not any(acq.Session & {'session_date': date})
        and all([x in os.listdir(rawPath + date + '/') for x in ['speedgoat','blackrock']])):
        
            # insert session
            acq.Session.insert1((date,MONKEY,RIG))
            
            # insert users
            acq.Session.User.insert1((date,MONKEY,'njm2149'))
            if date >= '2019-11-01':
                acq.Session.User.insert1((date,MONKEY,'emt2177'))

In [11]:
# aggregate pipelines
Pipeline = {'common': dj.ERD(acq) + dj.ERD(act) + dj.ERD(eph) + dj.ERD(lab) + dj.ERD(pro) + dj.ERD(ref),
            'full': dj.ERD(acq) + dj.ERD(act) + dj.ERD(eph) + dj.ERD(lab) + dj.ERD(pro) + dj.ERD(ref) + dj.ERD(pac),
            'pacman': dj.ERD(pac),
            'emg': (dj.ERD(acq.EmgChannelGroup)-1) + (dj.ERD(pro.MotorUnit)-1+10),
            'neural': (dj.ERD(acq.NeuralChannelGroup)-1) + (dj.ERD(pro.Neuron)-1+10)}

### Questions

In [ ]:
"""
best way to develop populate functions? (faster way of getting the next unpopulated key in jupyter vs. explicit database algebra?)
reminder of schema name types
"""